In [ ]:
import psycopg2
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime
import datetime
import time 


try:
    conn_ord = psycopg2.connect(dbname='123456', user='6789', password='00EFKJEWFJ', host='100.000.000.0')
    print('successful connection to orders database!')
except:
    print('connection to orders database error')

month = str(datetime.datetime.now().strftime("%b").upper())
orderlog=r'C:\daily_report\orderlog_'+month+'.csv'

#create function to get orderstat for a day:

def get_orders(df, date):
    start = time. time()
    cursor_ord = conn_ord.cursor()
    cursor_ord.execute('''  
    with
    orders as (     select orderid, clientid, memberid, timeentry, amountfilled, amountinitial, status, ordertype, dir,price, timeinforce, seqlast
                    from order_info 
                    where 	clientid not in ('FFFFFFF')  
                            and timeentry::date=%s
                            and amountfilled<100)

SELECT 			timeentry::date as date,
                clientid, 
				memberid,
				(case when timeentry::date between make_date(date_part('year', timeentry::date)::int, 3,11) and make_date(date_part('year', timeentry::date)::int, 11,4)
						then (case  when timeentry::time<'15:30' 	
                                    then 0
									else 1 end)
						else (case  when timeentry::time<'14:30' 	
                                    then 0
									else 1 end)
						end)													as is_evening,
				count( distinct case when amountfilled<100 	and status in (2,3) 	and ordertype=7 	then orderid end) 	as num_oddlots_filled_oob,
				count( distinct case when amountfilled<100 	and status in (2,3) 				        then orderid end) 	as num_oddlots_filled,
				count( distinct case when amountinitial<100 			            and ordertype=7 	then orderid end) 	as num_oddlots_oob,
				count( distinct case when amountinitial<100 							                then orderid end) 	as num_oddlots,
				count( distinct case when ordertype=7 								                    then orderid end) 	as num_oob, 
				count( distinct 										                                orderid) 		    as num,
				round(sum(CASE 	WHEN ordertype=7
						THEN  	0.1
							+0.0022*amountinitial
							+(CASE 	WHEN  dir::INT=2 
							THEN (CASE WHEN amountinitial*price*0.0000130 < 0.01 THEN  0.01 ELSE  amountinitial*price*0.0000130 end)
							ELSE 0 END)
							+(CASE 	WHEN  dir::INT=2 
								THEN (		CASE 	WHEN amountinitial*0.000119  > 5.95 
											THEN  5.95 
											ELSE  amountinitial*0.000119  END)   
								ELSE 0 END)
							+0.001197
							+0.331940430*amountinitial*price/1000000
							+2.36844405*amountinitial*price*0.25/1000000 
							+amountinitial *(	CASE WHEN timeinforce<>7 	THEN	0.6*0.003-0.4*0.002 
														ELSE 	0.0001		
										END) 
						ELSE 0 END),2)	as fees_all,
				round(sum(CASE 	WHEN ordertype=7 and amountinitial<100
						THEN  	0.1
							+0.0022*amountinitial
							+(CASE 	WHEN  dir::INT=2 
							THEN (CASE WHEN amountinitial*price*0.0000130 < 0.01 THEN  0.01 ELSE  amountinitial*price*0.0000130 end)
							ELSE 0 END)
							+(CASE 	WHEN  dir::INT=2 
								THEN (		CASE 	WHEN amountinitial*0.000119  > 5.95 
											THEN  5.95 
											ELSE  amountinitial*0.000119  END)   
								ELSE 0 END)
							+0.001197
							+0.331940430*amountinitial*price/1000000
							+2.36844405*amountinitial*price*0.25/1000000 
							+amountinitial *(	CASE WHEN timeinforce<>7 	THEN	0.6*0.003-0.4*0.002 
														ELSE 	0.0001		
										END) 
						ELSE 0 END),2)	as fees_odd		
  FROM 
  (     select a.orderid, clientid, memberid, timeentry, amountfilled, amountinitial, status, ordertype, dir,price, timeinforce
        from orders a
        join (select orderid, max(seq) as max from trade_action group by 1) ta2 on a.seqlast=ta2.max  
        where   ordertype<>7
        union 
        select orderid, clientid, memberid, timeentry, amountfilled, amountinitial, status, ordertype, dir,price, timeinforce
        FROM orders
        where   ordertype=7) b

group by 1,2,3,4
    ''', (date,))
    end = time. time()
    dur_= str(float(end-start)/60)
    orderd_ = pd.DataFrame(cursor_ord.fetchall())
    cursor_ord.close()
    df=pd.concat([df, orderd_], axis=0)
    print('{} has finished for {} minutes!'.format(date, dur_))
    return df
    


    
#download available dates in db
cursor_ord = conn_ord.cursor()
cursor_ord.execute('''  select distinct timeentry::date
                        from order_info
                        where timeentry::date<>current_date''')
dates_ord = pd.DataFrame(cursor_ord.fetchall())
cursor_ord.close()
db=dates_ord[0].tolist()
#upload available dates in log
try:
    order_=pd.read_csv(orderlog, sep=';',header=None,dtype='unicode')
    d=set(list(order_[0]))
    d=[datetime.strptime( x, '%Y-%m-%d').date() for x in d]
    print('Order Log parsed')
except:
    print('Orders Log file is empty!')
    d=[]
    order_=pd.DataFrame()

#match dates, get new available
db=[x for x in db if not x in d]

for t in db:
    order_=get_orders(order_, t)

    
order_.columns=['TradeDate', 'ClientCode', 'Broker',  'Is evening', 'Num Odds Fill OOB', 'Num Odds Fill', 'Num Odds OOB', 'Num Odds', 'Num OOB', 'Num', 'Fees All', 'Fees Odds']
order_ = order_.sort_values(by='TradeDate', ascending=False).reset_index(drop=True)

outfile=open(orderlog, 'w')
order_.to_csv(outfile, sep=';', header=True, index=False)

outfile.close()

#create file for the next month
if int(datetime.datetime.now().strftime("%d"))>25:
    month2= str(datetime.date(int(datetime.datetime.now().strftime("%Y")),datetime.datetime.now().month+1, 1).strftime("%b").upper())
    orderlog2=r'C:\daily_report\orderlog_'+month2+'.csv'
    outfile2=open(orderlog2, 'w')
    outfile2.close()
    

print('finish!')

